In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 6.6 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim
#from summarizer import Summarizer,TransformerSummarizer

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
path = "drive/MyDrive/dataset/cnn_stories_tokenized"
import os

stories = os.listdir(path)

In [ ]:
final_stories=[]
for story in stories:

  text = open(path+"/"+story,"r").read()
  final_stories.append(text)

In [ ]:
def text_process(text):

  text = text.replace("\n","")
  text = text.replace("''","")
  text = text.replace("``","")
  final_text = text.split("@highlight")
  data = final_text[0].split("- --")
  try:
    return data[1]
  except:
    return data[0]

In [ ]:
!pip install summarizers

In [ ]:
from summarizers import Summarizers
summ = Summarizers('normal', device=device)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
text_process(final_stories[15])

" PepsiCo Inc. will spell out that its Aquafina bottled water is made with tap water , a concession to the growing environmental and political opposition to the bottled water industry . According to Corporate Accountability International , a U.S. watchdog group , the world 's No. 2 beverage company will include the words  Public Water Source  on Aquafina labels .Pepsi 's Aquafina bottled water and Coca-Cola 's Dasani are made from purified tap water . If this helps clarify the fact that the water originates from public sources , then it 's a reasonable thing to do ,  said Michelle Naughton , a Pepsi-Cola North America spokeswoman .Pepsi Chief Executive Indra Nooyi told Reuters earlier this week the company was considering such a move .Pepsi 's Aquafina and Coca-Cola Co 's Dasani are both made from purified water sourced from public reservoirs , as opposed to Danone 's Evian or Nestle 's Poland Spring , so-called  spring waters ,  shipped from specific locations the companies say have n

In [ ]:
dataset = []

for data in final_stories:

  data = text_process(data)
  #data = summ(data)
  dataset.append(data)

In [ ]:
avg_length = sum([len(data.split()) for data in dataset])/len(dataset)
avg_length

709.5714285714286

In [ ]:
max_length=100

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
model_path = "drive/MyDrive/dataset/saved_model.pt"

In [ ]:
torch.save(model.state_dict(),model_path)

In [ ]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.00001)

In [ ]:
tokenizer.encode(" TL;DR ")

[24811, 26, 7707, 220]

In [ ]:
extra_length = len(tokenizer.encode(" TL;DR ")) 

In [ ]:
class ReviewDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
summary_dataset = ReviewDataset(tokenizer, dataset, max_length)

In [ ]:
dataloader = DataLoader(summary_dataset, batch_size=40, shuffle=True, drop_last=True)

In [ ]:
len(dataloader)

87

In [ ]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to(device)
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 2 == 0:
                    print("loss: %f, %d"%(loss, idx))

In [ ]:
train(model=model, optimizer=optimizer, dl=dataloader, epochs=1)

loss: 3.771079, 0
loss: 3.631087, 2
loss: 3.764482, 4
loss: 3.555856, 6
loss: 3.624529, 8
loss: 3.552516, 10
loss: 3.746445, 12
loss: 3.567638, 14
loss: 3.751462, 16
loss: 3.566634, 18
loss: 3.546463, 20
loss: 3.509734, 22
loss: 3.426100, 24
loss: 3.355713, 26
loss: 3.300899, 28
loss: 3.580971, 30
loss: 3.447636, 32
loss: 3.464638, 34
loss: 3.463646, 36
loss: 3.510619, 38
loss: 3.306430, 40
loss: 3.396185, 42
loss: 3.371356, 44
loss: 3.443477, 46
loss: 3.194968, 48
loss: 3.299978, 50
loss: 3.247885, 52
loss: 3.226656, 54
loss: 3.430506, 56
loss: 3.396808, 58
loss: 3.361495, 60
loss: 3.352969, 62
loss: 3.295448, 64
loss: 3.422029, 66
loss: 3.234206, 68
loss: 3.300319, 70
loss: 3.385775, 72
loss: 3.265890, 74
loss: 3.312806, 76
loss: 3.269865, 78
loss: 3.237508, 80
loss: 3.252749, 82
loss: 3.276454, 84
loss: 3.255768, 86


In [ ]:
def topk(probs, n=9):
    
    probs = torch.softmax(probs, dim= -1)
    
    
    tokensProb, topIx = torch.topk(probs, k=n)
    
    
    tokensProb = tokensProb / torch.sum(tokensProb)

    
    tokensProb = tokensProb.cpu().detach().numpy()

    
    choice = np.random.choice(n, 1, p = tokensProb)
    tokenId = topIx[choice][0]

    return int(tokenId)

In [ ]:
def model_infer(model, tokenizer, review, max_length=100):
    # Preprocess the init token (task designator)
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to(device)

    with torch.set_grad_enabled(False):
        
        output = model(initial_input)

        
        logits = output.logits[0,-1]

        # Make a top-k choice and append to the result
        result.append(topk(logits))

        # For max_length times:
        for _ in range(max_length):
            
            input = torch.tensor(result).unsqueeze(0).to(device)
            output = model(input)
            logits = output.logits[0,-1]
            res_id = topk(logits)

            
            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else: 
                result.append(res_id)
    
    return tokenizer.decode(result)

In [ ]:
sample_reviews = [data.split(" TL;DR ")[0] for data in random.sample(dataset, 1)]
sample_reviews

[" Chelsea stunned titleholders Barcelona with a 1-0 victory in the first leg of their European Champions League semifinal on WednesdayIvory Coast international Didier Drogba scored the only goal of the game late in first-half injury time at a rain-swept Stamford Bridge in west London thanks to a rare mistake by Barcelona 's Lionel Messi .After watching Spanish league leaders Real Madrid lose to Bayern Munich the night before , Barcelona had hoped to avoid any slip-ups as they attempted to defend their title .And the game appeared to be going their way as they dominated possession with Messi creating chances whenever he got on the ball . But the visitors were profligate in front of goal with Cesc Fabregas particularly guilty , the Spain international missing a hat-trick of clear-cut chances .As halftime approached Chelsea grew in confidence having weathered the initial storm and took a shock lead with the very last kick of the opening period .Messi 's rare lapse in concentration allowe

In [ ]:
for review in sample_reviews:
    summaries = set()
    print(review)
    while len(summaries) <=1:
        summary = model_infer(model, tokenizer, review + " TL;DR ").split(" TL;DR ")[1].strip()
        if summary not in summaries:
            summaries.add(summary)
    print("Summaries: "+ str(summaries) +"\n")

 Chelsea stunned titleholders Barcelona with a 1-0 victory in the first leg of their European Champions League semifinal on WednesdayIvory Coast international Didier Drogba scored the only goal of the game late in first-half injury time at a rain-swept Stamford Bridge in west London thanks to a rare mistake by Barcelona 's Lionel Messi .After watching Spanish league leaders Real Madrid lose to Bayern Munich the night before , Barcelona had hoped to avoid any slip-ups as they attempted to defend their title .And the game appeared to be going their way as they dominated possession with Messi creating chances whenever he got on the ball . But the visitors were profligate in front of goal with Cesc Fabregas particularly guilty , the Spain international missing a hat-trick of clear-cut chances .As halftime approached Chelsea grew in confidence having weathered the initial storm and took a shock lead with the very last kick of the opening period .Messi 's rare lapse in concentration allowed 

In [ ]:
s=""
while(len(summaries)>0):
  s = s + str(summaries.pop())

In [ ]:
summ = s.replace("<|endoftext|>","").replace("\n","")

In [ ]:
summ

"Chelsea's goalReal Madrid's Lionel Messi's mistake, the last time Real Madrid's Lionel Messi scored a goal in the final,"

In [ ]:
body = sample_reviews[0]
body

" Chelsea stunned titleholders Barcelona with a 1-0 victory in the first leg of their European Champions League semifinal on WednesdayIvory Coast international Didier Drogba scored the only goal of the game late in first-half injury time at a rain-swept Stamford Bridge in west London thanks to a rare mistake by Barcelona 's Lionel Messi .After watching Spanish league leaders Real Madrid lose to Bayern Munich the night before , Barcelona had hoped to avoid any slip-ups as they attempted to defend their title .And the game appeared to be going their way as they dominated possession with Messi creating chances whenever he got on the ball . But the visitors were profligate in front of goal with Cesc Fabregas particularly guilty , the Spain international missing a hat-trick of clear-cut chances .As halftime approached Chelsea grew in confidence having weathered the initial storm and took a shock lead with the very last kick of the opening period .Messi 's rare lapse in concentration allowed

In [ ]:
!pip install summarizers

In [ ]:
from summarizers import Summarizers

In [ ]:
sum = Summarizers()

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
test_sum = sum(body)
test_sum

'Chelsea stunned titleholders Barcelona with a 1-0 victory in the first leg of their European Champions League semifinal on Wednesday.'

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [ ]:
scores = scorer.score(test_sum, summ)
scores

{'rouge1': Score(precision=0.17391304347826086, recall=0.19047619047619047, fmeasure=0.1818181818181818)}

In [ ]:
pip install langdetect

     |████████████████████████████████| 981 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=e283b8c54b8ac1c099580d87446bec82e85d15d7453332d9f3b1136d27f425a8
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
from langdetect import detect

In [ ]:
detect("Chelsea stunned titleholders Barcelona with a 1-0 victory in the first leg of their European Champions League semifinal on WednesdayIvory Coast international Didier Drogba scored the only goal of the game late in first-half injury time at a rain-swept Stamford Bridge in west London thanks to a rare mistake by Barcelona 's Lionel Messi .After watching Spanish league leaders Real Madrid lose to Bayern Munich the night before , Barcelona had hoped to avoid any slip-ups as they attempted to defend their title .And the game appeared to be going their way as they dominated possession with Messi creating chances whenever he got on the ball . But the visitors were profligate in front of goal with Cesc Fabregas particularly guilty , the Spain international missing a hat-trick of clear-cut chances .As halftime approached Chelsea grew in confidence having weathered the initial storm and took a shock lead with the very last kick of the opening period .Messi 's rare lapse in concentration allowed Brazilian midfielder Ramires to career down the left wing .With few options in front of him Ramires fired a low ball across the penalty box which eluded Barcelona 's defense before somehow finding Drogba lurking at the back post .Barcelona goalkeeper Victor Valdes could do little but palm the striker 's first-time shot into the back of the net .Despite Barcelona experiencing the rare feeling of being behind at halftime , it was Chelsea that grew more comfortable as the game progressed , snuffing out the Catalan club 's attacks with increasing confidence .Yet deep in injury time Barcelona could have leveled the tie and score a crucial away goal , but substitute Pedro hit the post with his low curling shot and Sergio Busquets blazed the rebound over the bar .Chelsea -- controversially knocked out of the competition by a late Barcelona goal in 2009 on the same ground -- held on for a famous victory . It was a great result . When you play Barcelona you have to do a lot of defending , just try and limit their threat and be clinical with your chances ,  Chelsea 's interim coach Roberto Di Matteo told British network ITV .And clinical they were . Drogba 's goal turned out to be Chelsea 's only shot on target in a game where Barcelona enjoyed the majority of possession : Chelsea completed 194 passes while Barcelona managed 782 , almost four times as many .Yet the one statistic that counts is the score and for those that played in the last Chelsea-Barcelona Champions League semifinal in 2009 -- where Chelsea had five clear penalty claims turned down before an injury-time Andres Iniesta goal clinched their place in the final -- the victory was sweet revenge . I think we 've learned , we 've improved from three years ago . Now we 're going to go there , have a go and try to qualify -LRB- for the final -RRB- ,  said Drogba .Now attention turns to the second leg at Barcelona 's Nou Camp . It 's going to be tough there ,  admitted Drogba .  They will have opportunities . We respect them . But we 're not in awe of them ")

'en'

In [ ]:
pip install translate

In [ ]:
from translate import Translator

In [ ]:
translator= Translator(to_lang="bengali")
translation = translator.translate("Chelsea plays")

In [ ]:
print(translation)

কি  খাবার খেলে
